# Chilean Growth Slowdown Analysis

This notebook demonstrates how to use the replication code for Toni, Paniagua & Ordenes (2023).

**Paper:** Policy Changes and Growth Slowdown: Assessing the Lost Decade of the Latin American Miracle

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import project modules
import sys
sys.path.insert(0, '..')

from src.config import DONOR_POOL_GROUP_II, TREATMENT_YEAR, RANDOM_SEED
from src.data_loader import assemble_panel_data
from src.synthetic_control import fit_synthetic_control, placebo_test
from src.causal_impact import fit_bsts_model, summarize_impact
from src.visualization import plot_scm_main, plot_scm_trend

# Set random seed
np.random.seed(RANDOM_SEED)

## 1. Load Data

Load panel data from World Bank, Penn World Table, and other sources.

In [ ]:
# Load or fetch data
df = assemble_panel_data()

print(f"Panel data shape: {df.shape}")
print(f"Countries: {df['country'].nunique()}")
print(f"Years: {df['year'].min()} - {df['year'].max()}")
df.head()

## 2. Synthetic Control Method

Fit SCM to estimate what Chile's GDP would have been without the 2014 policy changes.

In [ ]:
# Fit synthetic control
result = fit_synthetic_control(
    df=df,
    outcome_var="gdp_per_capita",
    treated_unit="CHL",
    treatment_year=TREATMENT_YEAR,
    donor_pool=DONOR_POOL_GROUP_II,
)

# Display weights
print("Optimal Weights:")
print(result.weights[result.weights > 0.01].sort_values(ascending=False))

In [ ]:
# Model fit statistics
print(f"Pre-treatment RMSPE:  {result.rmspe_pre:.2f}")
print(f"Post-treatment RMSPE: {result.rmspe_post:.2f}")
print(f"RMSPE Ratio:          {result.rmspe_ratio:.2f}")

In [ ]:
# Predictor balance
result.predictor_balance

## 3. Visualize Results

In [ ]:
# Plot main result
plot_scm_main(result)
plt.show()

In [ ]:
# Plot HP-filtered trends
plot_scm_trend(result)
plt.show()

In [ ]:
# Treatment effect
final_year = result.actual.index[-1]
gap = result.gap.loc[final_year]
pct = (gap / result.synthetic.loc[final_year]) * 100

print(f"Treatment Effect ({final_year}):")
print(f"  Actual Chile:    ${result.actual.loc[final_year]:,.0f}")
print(f"  Synthetic Chile: ${result.synthetic.loc[final_year]:,.0f}")
print(f"  Gap:             ${gap:,.0f} ({pct:.1f}%)")

## 4. BSTS Analysis

Bayesian Structural Time Series for causal inference with uncertainty quantification.

In [ ]:
# Fit BSTS model
bsts_result = fit_bsts_model(
    df=df,
    outcome_var="gdp_per_capita",
    treated_unit="CHL",
    treatment_year=TREATMENT_YEAR,
)

# Print summary
print(summarize_impact(bsts_result))

## 5. Conclusion

The analysis shows that Chile's growth slowdown after 2014 resulted in approximately 10% lower GDP per capita by 2019 compared to the synthetic counterfactual.